In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
sys.path.append('/app')

In [3]:
from pydantic_models import MeetingExtraction, EntityExtraction, MeetingNameAndSummary
import pandas as pd
from vexa import VexaAPI

In [4]:
from psql_helpers import get_session

In [5]:
from search import SearchAssistant

In [6]:
search_assistant = SearchAssistant()
await search_assistant.initialize()


12:18:09 - vexa_api - INFO - Initializing SearchAssistant


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

In [7]:
from psql_helpers import get_token_by_email
token,_ = await get_token_by_email('dmitrygrankin0@gmail.com')

In [8]:
from api_client import APIClient
api = APIClient(email='dmitrygrankin0@gmail.com')
meetings = await api.get_meetings()

In [9]:
meeting_id = meetings['meetings'][0]['meeting_id']

In [10]:
await search_assistant.clean_meeting_data(meeting_id)

(True, True)

In [17]:
vexa_api = VexaAPI(token=api.token)
user_id = (await vexa_api.get_user_info())['id']
transcription = await vexa_api.get_transcription(meeting_session_id=meeting_id, use_index=True)
df, formatted_input, start_time, _, transcript = transcription
df

Vexa token: 918c9f3899d54f3bbcf445cd053c2304
User information retrieved successfully.


,index,speaker,speaker_id,content,html_content,html_content_short,keywords,timestamp,formatted_time,time_tuple,initials,chunk_number
0,0,Dmitriy Grankin,TBD,"to attend a future proof meeting, so sorry ab...",I apologize for not being able to attend the <...,Future Proof,[],2024-11-30T11:31:34.830000Z,00:00,"(0, 0)",DG,0
1,1,Dmitriy Grankin,TBD,"Good as gold, thanks Jim. So should we includ...","Good as gold, thanks <b>Jim</b>. Should we inc...",Jim,[],2024-11-30T11:31:42.930000Z,00:08,"(0, 8)",DG,0
2,2,Dmitriy Grankin,TBD,I'd like to move please. Happy to move Mike. ...,"I'd like to <b>move</b>, please. Happy to move...","move, Mike, Lou, Susan",[],2024-11-30T11:31:51.116000Z,00:16,"(0, 16)",DG,0
3,3,Dmitriy Grankin,TBD,"Contrary no, it's carried. Disclosure of memb...","<b>Contrary</b> to what was stated, it's carri...","Contrary, disclosure, members' interests",[],2024-11-30T11:31:56.156000Z,00:21,"(0, 21)",DG,0
4,4,Dmitriy Grankin,TBD,do we have any today no i'll move on to item ...,Do we have any <b>late items</b> today? No? I'...,"late items, item four, late items, Ken",[],2024-11-30T11:32:14.056000Z,00:39,"(0, 39)",DG,1
5,5,Dmitriy Grankin,TBD,I don't see any need for change. We do have a...,I don't see any need for change. We do have a ...,guests,[],2024-11-30T11:32:21.686000Z,00:46,"(0, 46)",DG,1
6,6,Dmitriy Grankin,TBD,meeting today um but they're well aware of th...,"In the meeting today, everyone is well aware o...","Andrew Graham, Susan, aye",[],2024-11-30T11:32:39.178000Z,01:04,"(1, 4)",DG,1
7,7,Dmitriy Grankin,TBD,Aye.,Aye.,,[],2024-11-30T11:32:40.058000Z,01:05,"(1, 5)",DG,1
8,8,Dmitriy Grankin,TBD,Ms Cormack Aye Mr Leitch Aye Ms Gormack Aye M...,"<b>Ms. Cormack</b> Aye, <b>Mr. Leitch</b> Aye,...","Ms. Cormack, Mr. Leitch, Ms. Gormack, Mr. Leit...",[],2024-11-30T11:32:59.342000Z,01:24,"(1, 24)",DG,1
9,9,Dmitriy Grankin,TBD,Andrew Graham speaking. My diary's showing me...,<b>Andrew Graham</b> speaking. My diary shows ...,Andrew Graham,[],2024-11-30T11:33:06.978000Z,01:32,"(1, 32)",DG,1


In [33]:
discussion_points_df = await MeetingExtraction.extract(formatted_input)

In [34]:
discussion_points_df

,item,type,summary,details,speaker
0,Overview of AI Marketing Directory's services,insight,AI Marketing Directory is a curated directory ...,The company focuses on a niche market with a b...,AMD
1,Discussion on influencer marketing strategy,strategy,The strategy involves creating content that in...,"Influencers are compensated to repost content,...",AMD
2,Examples of successful and unsuccessful campaigns,feedback,The meeting highlighted both successful and un...,"Successful example: Hautech, an AI fashion des...",AMD
3,Proposal for collaboration with Vexa,proposal,"A proposal was made to collaborate with Vexa, ...",The offer includes a monthly fee of $149 for a...,AMD
4,Details on pricing and service offerings,budget discussion,"The pricing structure was discussed, including...","The service fee is now $149 per month, which i...",AMD
5,Next steps for communication and follow-up,follow-up item,The meeting concluded with plans for further c...,AMD will send a link to their website and serv...,AMD


In [35]:
discussion_points_df

,item,type,summary,details,speaker
0,Overview of AI Marketing Directory's services,insight,AI Marketing Directory is a curated directory ...,The company focuses on a niche market with a b...,AMD
1,Discussion on influencer marketing strategy,strategy,The strategy involves creating content that in...,"Influencers are compensated to repost content,...",AMD
2,Examples of successful and unsuccessful campaigns,feedback,The meeting highlighted both successful and un...,"Successful example: Hautech, an AI fashion des...",AMD
3,Proposal for collaboration with Vexa,proposal,"A proposal was made to collaborate with Vexa, ...",The offer includes a monthly fee of $149 for a...,AMD
4,Details on pricing and service offerings,budget discussion,"The pricing structure was discussed, including...","The service fee is now $149 per month, which i...",AMD
5,Next steps for communication and follow-up,follow-up item,The meeting concluded with plans for further c...,AMD will send a link to their website and serv...,AMD


In [36]:
topics_df = await EntityExtraction.extract(messages)

In [37]:
topics_df

,entity,type,summary,details,speaker
0,AI Marketing Directory,company,A company that operates a directory of AI busi...,AI Marketing Directory has co-branding subsidi...,AMD
1,Ethan Noah,person,A speaker in the meeting who discusses the ope...,Ethan Noah provides insights into the company'...,EN
2,Dmitry Grankin,person,A participant in the meeting who engages with ...,Dmitry Grankin asks questions about the effect...,DG
3,Twitter,platform,A social media platform where AI Marketing Dir...,The company has a large network of influencers...,AMD
4,AI business tools,topic,"The focus of AI Marketing Directory, which cur...",The directory includes a niche selection of AI...,AMD
5,influencers,person,Individuals with a significant following on so...,AI Marketing Directory collaborates with influ...,AMD
6,micro-influencers,person,Smaller-scale influencers who also participate...,These influencers are part of the network that...,AMD
7,click to ai,product,A tool promoted by AI Marketing Directory that...,The marketing strategy for this tool included ...,AMD
8,Hautech,company,A successful client of AI Marketing Directory ...,Hautech has been one of the more successful pr...,AMD
9,Quick Blocks,company,A client of AI Marketing Directory that has no...,Quick Blocks focuses on B2B services for healt...,AMD


In [38]:
discussion_points_df

,item,type,summary,details,speaker
0,Overview of AI Marketing Directory's services,insight,AI Marketing Directory is a curated directory ...,The company focuses on a niche market with a b...,AMD
1,Discussion on influencer marketing strategy,strategy,The strategy involves creating content that in...,"Influencers are compensated to repost content,...",AMD
2,Examples of successful and unsuccessful campaigns,feedback,The meeting highlighted both successful and un...,"Successful example: Hautech, an AI fashion des...",AMD
3,Proposal for collaboration with Vexa,proposal,"A proposal was made to collaborate with Vexa, ...",The offer includes a monthly fee of $149 for a...,AMD
4,Details on pricing and service offerings,budget discussion,"The pricing structure was discussed, including...","The service fee is now $149 per month, which i...",AMD
5,Next steps for communication and follow-up,follow-up item,The meeting concluded with plans for further c...,AMD will send a link to their website and serv...,AMD


In [129]:
topics_df

,entity,type,summary,details,speaker
0,ВЕКСа,продукт,"Продукт, который транскрибирует звонки и собир...",ВЕКСа позволяет пользователям получать доступ ...,DG
1,Product Hunt,платформа,Платформа для запуска и продвижения новых прод...,Обсуждение возможностей повторного запуска про...,ON
2,Бразилия,страна,"Страна, откуда продолжают приходить новые поль...",Обсуждение роста пользователей из Бразилии и и...,DG
3,Андрей,человек,"Участник обсуждения, связанный с историей о вс...","Обсуждение истории, связанной с Андреем и его ...",ON
4,Лиза,человек,"Участник обсуждения, который отправил встречу.","Лиза отправила встречу, которая продолжает под...",ON
5,Алексей,человек,"Дизайнер, который работал над продуктом.",Обсуждение дизайна продукта и его влияния на в...,DG
6,Дмитрий Гранкин,человек,"Один из спикеров, работающий над продуктом.",Обсуждение текущих задач и планов по развитию ...,DG
7,Ольга Немировская,человек,"Один из спикеров, работающий над продуктом.",Обсуждение текущих задач и обратной связи по п...,ON


In [130]:
discussion_points_df['model'] = 'MeetingExtraction'
topics_df['model'] = 'EntityExtraction'

for df, name_col in [(discussion_points_df, 'item'), (topics_df, 'entity')]:
    df.rename(columns={name_col: 'topic_name', 'type': 'topic_type'}, inplace=True)

points_df = pd.concat([discussion_points_df, topics_df]).reset_index(drop=True)
points_df = points_df.reset_index().rename(columns={'index': 'summary_index'})

final_df = points_df.groupby('summary_index').agg({
    'topic_name': 'first', 'topic_type': 'first', 'summary': 'first',
    'details': 'first', 'speaker': 'first', 'model': 'first'
}).reset_index()
final_df

,summary_index,topic_name,topic_type,summary,details,speaker,model
0,0,Проблемы с логином через разные аккаунты,проблема,Проблемы с доступом к системе из-за использова...,Пользователь не может войти через обычный Gmai...,Olga Nemirovskaya,MeetingExtraction
1,1,Обсуждение индексации встреч,обсуждение,"Индексация встреч происходит, но есть задержки.","Некоторые встречи не индексируются вовремя, чт...",Dmitriy Grankin,MeetingExtraction
2,2,Проблемы с платежами,проблема,Проблемы с подключением платежей.,Пользователь не смог подключить платежи из-за ...,Olga Nemirovskaya,MeetingExtraction
3,3,Обсуждение функционала дашборда,обсуждение,Обсуждение улучшений в дашборде.,"Дашборд был улучшен, но требуется дальнейшая д...",Dmitriy Grankin,MeetingExtraction
4,4,Планирование маркетинговых действий,план,Планирование маркетинговых действий после заве...,Необходимо заняться маркетингом после завершен...,Dmitriy Grankin,MeetingExtraction
5,5,Обсуждение пользователей и их активности,обсуждение,Обсуждение текущего количества пользователей и...,"На данный момент 16-17 активных пользователей,...",Dmitriy Grankin,MeetingExtraction
6,6,Проблемы с отображением дат в системе,проблема,Некорректное отображение дат в системе.,"Система неправильно парсит даты, что создает п...",Dmitriy Grankin,MeetingExtraction
7,7,Обсуждение возможности добавления новых функций,обсуждение,Обсуждение новых функций для улучшения работы ...,Необходимо добавить возможность фильтрации и т...,Olga Nemirovskaya,MeetingExtraction
8,8,ВЕКСа,продукт,"Продукт, который транскрибирует звонки и собир...",ВЕКСа позволяет пользователям получать доступ ...,DG,EntityExtraction
9,9,Product Hunt,платформа,Платформа для запуска и продвижения новых прод...,Обсуждение возможностей повторного запуска про...,ON,EntityExtraction


In [163]:
%reload_ext autoreload

In [164]:
meeting_summary = await MeetingNameAndSummary.extract(
        formatted_input, 
        final_df.drop(columns=['model']).to_markdown(index=False)
    )

In [165]:
len(meeting_summary.meeting_name)

39

In [166]:
meeting_summary.meeting_name

'Проблемы с логином и индексацией встреч'

In [153]:
formatted_input

'Meeting Metadata:\nStart Date and Time: 2024-11-15 14:32:30\nSpeakers (Always  reference the names EXACTLY as provided with NO changes letter by letter):\n  DG: Dmitriy Grankin\n  ON: Olga Nemirovskaya\n\nMeeting Transcript:\n\nON (0):  Да, я все раскупировала, но вот оно мне выдавило манифест.\nON (1):  Понятно, ну ладно.\nDG (2):  Понятно. Ну, ладно. Там не такие существенные изменения в самой станции.\nDG (3):  Ага.\nDG (4):  Вот. Я дашборд.\nDG (5):  прокачал он классно вот и он там у тебя почти сейчас должен пойти на индексацию когда тебя увидел там твои все твои предыдущие встречи обрабатывать начнет вот у меня получается\nON (6):  Проблема в том, что я, видишь, с разных аккаунтов логинюсь, и иногда я не могу заходить через обычный свой Gmail, и вот это, конечно, очень неудобно, я пока так и не научилась нормально переключаться типа мне надо тогда все\nDG (7):  чистить и вообще все снасть ну вот я этот вопрос как раз решал ага я сделал Я сделал логин, логаут на дешборде, на экст

In [167]:
meeting_summary.summary

'Обсуждение **проблем с логином** через разные аккаунты, что вызывает **неудобства** при переключении между ними. **Индексация встреч** происходит, но с **задержками**, что затрудняет доступ к информации. Также обсуждены **проблемы с платежами** из-за отсутствия счета, **некорректное отображение дат** в системе, и необходимость **добавления новых функций** для улучшения работы системы. Обсуждение **планирования маркетинговых действий** после завершения разработки и текущего количества **пользователей** (16-17 активных). Обсуждение **возможностей запуска продукта** на **Product Hunt** и роста пользователей из **Бразилии**.'

In [3]:
from chat import MeetingContextProvider

ModuleNotFoundError: No module named 'chat'

In [4]:
from psql_helpers import get_session,get_meeting_by_id

ModuleNotFoundError: No module named 'psql_helpers'

In [185]:
async with get_session() as session:
        context_provider = MeetingContextProvider()
        context = await context_provider.get_context(session=session,meeting_id= [meeting_id])


DBAPIError: (sqlalchemy.dialects.postgresql.asyncpg.Error) <class 'asyncpg.exceptions.DataError'>: invalid input for query argument $1: ['7973854d-ce5a-4263-b0f5-74007ea7a630'] ('list' object has no attribute 'bytes')
[SQL: SELECT meetings.id, meetings.meeting_id, meetings.transcript, meetings.timestamp, meetings.meeting_name, meetings.meeting_summary, meetings.is_indexed, meetings.last_update, speakers_1.id AS id_1, speakers_1.name, discussion_points_1.id AS id_2, discussion_points_1.summary_index, discussion_points_1.summary, discussion_points_1.details, discussion_points_1.referenced_text, discussion_points_1.meeting_id AS meeting_id_1, discussion_points_1.speaker_id, discussion_points_1.topic_name, discussion_points_1.topic_type, discussion_points_1.model, user_meetings_1.id AS id_3, user_meetings_1.meeting_id AS meeting_id_2, user_meetings_1.user_id, user_meetings_1.access_level, user_meetings_1.created_at, user_meetings_1.created_by, user_meetings_1.is_owner 
FROM meetings LEFT O

In [1]:
print(context)

NameError: name 'context' is not defined

In [188]:
async with get_session() as session:
    meeting = await get_meeting_by_id(session, meeting_id)


In [189]:
meeting.transcript